# 🏡 Zurich Real Estate Price Prediction
## Machine Learning Model Development

In diesem Notebook entwickeln und evaluieren wir Machine Learning Modelle zur Vorhersage von Immobilienpreisen in Zürich.

In [ ]:
# Benötigte Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import os

# Plots konfigurieren
plt.style.use('seaborn-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Daten laden

Wir laden den vorbereiteten Datensatz aus der explorativen Analyse.

In [ ]:
# Verarbeiteten Datensatz laden
df = pd.read_csv('../data/processed/modell_input_final.csv')

print(f"Datensatz: {df.shape[0]} Zeilen, {df.shape[1]} Spalten")
df.head()

In [ ]:
# Datenstatistik prüfen
df.describe()

## 2. Feature Auswahl und Datenaufbereitung

Wir identifizieren relevante Features und bereiten die Daten für das Modelltraining vor.

In [ ]:
# Feature und Zielwerte definieren
# Features: Quartier (als Code), Zimmeranzahl, Preisniveau, Baujahr, etc.
# Ziel: MedianPreis
X = df.drop(['MedianPreis', 'Quartier', 'Zimmeranzahl'], axis=1)
y = df['MedianPreis']

# Kategoriale und numerische Features identifizieren
cat_features = ['Quartier_Code']
num_features = [col for col in X.columns if col not in cat_features]

print("Kategoriale Features:", cat_features)
print("Numerische Features:", num_features)

In [ ]:
# Train-Test-Split (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Trainingsdaten: {X_train.shape[0]} Einträge")
print(f"Testdaten: {X_test.shape[0]} Einträge")

## 3. Baseline-Modelle

Wir implementieren einfache Baseline-Modelle zur Preisvorhersage.

In [ ]:
# Funktion zur Modellauswertung
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    print(f"MAE: {mae:.2f} CHF")
    print(f"RMSE: {rmse:.2f} CHF")
    print(f"R²: {r2:.4f}")
    
    # Visualisierung: Vorhergesagte vs. tatsächliche Werte
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.6)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel('Tatsächlicher Preis (CHF)', fontsize=12)
    plt.ylabel('Vorhergesagter Preis (CHF)', fontsize=12)
    plt.title('Vorhergesagte vs. Tatsächliche Preise', fontsize=14)
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    return mae, rmse, r2

In [ ]:
# Preprocessing-Pipeline erstellen
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ])

# 1. Lineare Regression als Baseline
lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

print("Training Lineare Regression...")
lr_pipeline.fit(X_train, y_train)

print("\nLineare Regression Evaluation:")
lr_metrics = evaluate_model(lr_pipeline, X_test, y_test)

In [ ]:
# 2. Random Forest als robustes Ensemble-Modell
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

print("Training Random Forest...")
rf_pipeline.fit(X_train, y_train)

print("\nRandom Forest Evaluation:")
rf_metrics = evaluate_model(rf_pipeline, X_test, y_test)

In [ ]:
# 3. Gradient Boosting für möglicherweise bessere Performance
gb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(n_estimators=100, random_state=42))
])

print("Training Gradient Boosting...")
gb_pipeline.fit(X_train, y_train)

print("\nGradient Boosting Evaluation:")
gb_metrics = evaluate_model(gb_pipeline, X_test, y_test)

## 4. Modellvergleich

Wir vergleichen die Performance der verschiedenen Modelle.

In [ ]:
# Metriken aller Modelle vergleichen
models = ['Lineare Regression', 'Random Forest', 'Gradient Boosting']
metrics = [lr_metrics, rf_metrics, gb_metrics]

# Metriken in DataFrame organisieren
metrics_df = pd.DataFrame({
    'Modell': models,
    'MAE (CHF)': [m[0] for m in metrics],
    'RMSE (CHF)': [m[1] for m in metrics],
    'R²': [m[2] for m in metrics]
})

metrics_df

In [ ]:
# Visualisierung der Modellperformance
plt.figure(figsize=(12, 5))

# MAE und RMSE
plt.subplot(1, 2, 1)
bar_width = 0.35
x = np.arange(len(models))
plt.bar(x - bar_width/2, metrics_df['MAE (CHF)'], bar_width, label='MAE')
plt.bar(x + bar_width/2, metrics_df['RMSE (CHF)'], bar_width, label='RMSE')
plt.xticks(x, models, rotation=45, ha='right')
plt.ylabel('Fehler (CHF)')
plt.title('MAE und RMSE nach Modell')
plt.legend()

# R²
plt.subplot(1, 2, 2)
plt.bar(models, metrics_df['R²'], color='seagreen')
plt.ylim(0, 1)
plt.ylabel('R²')
plt.title('R² nach Modell')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

## 5. Cross-Validation

Wir führen eine Kreuzvalidierung durch, um die Stabilität der Modelle zu prüfen.

In [ ]:
# 5-fache Kreuzvalidierung für alle Modelle
print("Cross-Validation wird durchgeführt (5-fold)...")

# Negative MSE als Scoring-Metrik (wird später in RMSE umgewandelt)
lr_cv_scores = cross_val_score(lr_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
rf_cv_scores = cross_val_score(rf_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
gb_cv_scores = cross_val_score(gb_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')

# Umwandlung in RMSE
lr_cv_rmse = np.sqrt(-lr_cv_scores)
rf_cv_rmse = np.sqrt(-rf_cv_scores)
gb_cv_rmse = np.sqrt(-gb_cv_scores)

# Ergebnisse ausgeben
print(f"Lineare Regression - Mittlerer RMSE: {lr_cv_rmse.mean():.2f} CHF, Std: {lr_cv_rmse.std():.2f}")
print(f"Random Forest - Mittlerer RMSE: {rf_cv_rmse.mean():.2f} CHF, Std: {rf_cv_rmse.std():.2f}")
print(f"Gradient Boosting - Mittlerer RMSE: {gb_cv_rmse.mean():.2f} CHF, Std: {gb_cv_rmse.std():.2f}")

In [ ]:
# Boxplot der CV-Ergebnisse
plt.figure(figsize=(10, 6))
cv_data = [lr_cv_rmse, rf_cv_rmse, gb_cv_rmse]
plt.boxplot(cv_data, labels=models)
plt.ylabel('RMSE (CHF)')
plt.title('Cross-Validation Ergebnisse (5-fold)')
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

## 6. Feature Importance

Wir analysieren, welche Features den größten Einfluss auf die Preisvorhersage haben.

In [ ]:
# Feature Importance aus dem Random Forest Modell extrahieren
rf_model = rf_pipeline.named_steps['regressor']
preprocessor = rf_pipeline.named_steps['preprocessor']

# Transformierte Feature-Namen abrufen
ohe = preprocessor.named_transformers_['cat']
cat_feature_names = ohe.get_feature_names_out(cat_features)
feature_names = np.concatenate([num_features, cat_feature_names])

# Feature Importance extrahieren
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

# Top 15 Features anzeigen
n_top_features = min(15, len(feature_names))
plt.figure(figsize=(10, 8))
plt.title('Top 15 Feature Importance')
plt.barh(range(n_top_features), importances[indices[:n_top_features]], align='center')
plt.yticks(range(n_top_features), feature_names[indices[:n_top_features]])
plt.xlabel('Relative Wichtigkeit')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 7. Hyperparameter-Tuning

Wir optimieren das beste Modell durch Hyperparameter-Tuning.

In [ ]:
# Basierend auf den bisherigen Ergebnissen wählen wir das beste Modell für das Tuning
# (Annahme: Gradient Boosting, kann je nach tatsächlichen Ergebnissen variieren)
best_base_model = 'GradientBoosting'  # oder 'RandomForest', basierend auf CV-Ergebnissen

if best_base_model == 'RandomForest':
    # Hyperparameter-Grid für Random Forest
    param_grid = {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__max_depth': [None, 10, 20, 30],
        'regressor__min_samples_split': [2, 5, 10],
        'regressor__min_samples_leaf': [1, 2, 4]
    }
    
    # Pipeline mit RandomForest
    tuning_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(random_state=42))
    ])
    
else:  # GradientBoosting
    # Hyperparameter-Grid für Gradient Boosting
    param_grid = {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__learning_rate': [0.01, 0.1, 0.2],
        'regressor__max_depth': [3, 5, 7],
        'regressor__min_samples_split': [2, 5, 10]
    }
    
    # Pipeline mit GradientBoosting
    tuning_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', GradientBoostingRegressor(random_state=42))
    ])

# GridSearchCV mit 3-fold CV für schnelleres Tuning
print(f"Hyperparameter-Tuning für {best_base_model} wird durchgeführt...")
grid_search = GridSearchCV(
    tuning_pipeline, 
    param_grid, 
    cv=3, 
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Da das Tuning lang dauern kann, kommentieren wir es aus
# Entfernen Sie den Kommentar, wenn Sie bereit sind, es auszuführen
# grid_search.fit(X_train, y_train)

# # Beste Parameter anzeigen
# print(f"Beste Parameter: {grid_search.best_params_}")
# print(f"Bester RMSE: {np.sqrt(-grid_search.best_score_):.2f} CHF")

# # Bestes Modell evaluieren
# best_model = grid_search.best_estimator_
# print("\nEvaluation des optimierten Modells:")
# best_metrics = evaluate_model(best_model, X_test, y_test)

## 8. Modell speichern

Wir speichern das beste Modell für die Verwendung in der Streamlit-App.

In [ ]:
# Bestes Modell auswählen (ohne GridSearch)
# Wir verwenden das Modell mit dem besten R² aus unserem Vergleich
model_scores = [lr_metrics[2], rf_metrics[2], gb_metrics[2]]
best_model_index = np.argmax(model_scores)
best_model_name = models[best_model_index]

if best_model_index == 0:
    best_model = lr_pipeline
elif best_model_index == 1:
    best_model = rf_pipeline
else:
    best_model = gb_pipeline

print(f"Bestes Modell basierend auf R²: {best_model_name}")

# Modell-Verzeichnis erstellen, falls es nicht existiert
if not os.path.exists('../models'):
    os.makedirs('../models')

# Modell speichern
model_path = '../models/price_model.pkl'
with open(model_path, 'wb') as file:
    pickle.dump(best_model, file)

print(f"Modell wurde in {model_path} gespeichert.")

# Quartier-Mapping speichern für die App
quartier_mapping = dict(zip(df['Quartier_Code'], df['Quartier']))
mapping_path = '../models/quartier_mapping.pkl'
with open(mapping_path, 'wb') as file:
    pickle.dump(quartier_mapping, file)

print(f"Quartier-Mapping wurde in {mapping_path} gespeichert.")

## 9. Manuelle Vorhersagetest

Wir testen das Modell mit einigen Beispieleingaben.

In [ ]:
# Beispieleingaben erstellen
test_entries = [
    # Quartier Seefeld, 3 Zimmer, Baujahr 2010
    {'Quartier': 'Seefeld', 'Zimmeranzahl': 3, 'Baujahr': 2010},
    # Quartier Altstetten, 4 Zimmer, Baujahr 1980
    {'Quartier': 'Altstetten', 'Zimmeranzahl': 4, 'Baujahr': 1980},
    # Quartier Hottingen, 5 Zimmer, Baujahr 2000
    {'Quartier': 'Hottingen', 'Zimmeranzahl': 5, 'Baujahr': 2000}
]

# Quartier-Code-Mapping erstellen (umgekehrt)
inv_quartier_mapping = {v: k for k, v in quartier_mapping.items()}

# Funktionen für die Vorverarbeitung
def preprocess_test_entry(entry, df):
    """Bereitet einen Testeintrag für die Modellvorhersage vor"""
    # Quartier-Code abrufen
    quartier_code = inv_quartier_mapping.get(entry['Quartier'])
    zimmeranzahl = entry['Zimmeranzahl']
    baujahr = entry['Baujahr']
    
    # Durchschnittswerte für das Quartier finden
    quartier_data = df[df['Quartier'] == entry['Quartier']]
    if len(quartier_data) > 0:
        quartier_preisniveau = quartier_data['Quartier_Preisniveau'].mean()
        mediapreis_baualter = quartier_data['MedianPreis_Baualter'].mean()
    else:
        # Fallback: Durchschnittswerte aller Quartiere verwenden
        quartier_preisniveau = df['Quartier_Preisniveau'].mean()
        mediapreis_baualter = df['MedianPreis_Baualter'].mean()
    
    # Testeintrag als DataFrame erstellen
    test_df = pd.DataFrame({
        'Quartier_Code': [quartier_code],
        'Zimmeranzahl_num': [zimmeranzahl],
        'PreisProQm': [quartier_preisniveau * 10000],  # Approximation
        'MedianPreis_Baualter': [mediapreis_baualter],
        'Durchschnitt_Baujahr': [baujahr],
        'Preis_Verhältnis': [1.0],  # Standardwert, wird durch das Modell angepasst
        'Quartier_Preisniveau': [quartier_preisniveau]
    })
    
    return test_df

# Vorhersagen für die Testeinträge
print("Testvorhersagen mit dem besten Modell:")
for i, entry in enumerate(test_entries):
    test_df = preprocess_test_entry(entry, df)
    predicted_price = best_model.predict(test_df)[0]
    
    print(f"\nTest {i+1}: {entry['Quartier']}, {entry['Zimmeranzahl']} Zimmer, Baujahr {entry['Baujahr']}")
    print(f"Vorhergesagter Preis: {predicted_price:,.2f} CHF")
    
    # Tatsächlichen Durchschnittspreis für dieses Quartier und diese Zimmeranzahl finden
    similar_properties = df[
        (df['Quartier'] == entry['Quartier']) & 
        (df['Zimmeranzahl_num'] == entry['Zimmeranzahl'])
    ]
    
    if len(similar_properties) > 0:
        actual_avg = similar_properties['MedianPreis'].mean()
        print(f"Durchschnittlicher Preis im Datensatz: {actual_avg:,.2f} CHF")
        print(f"Differenz: {predicted_price - actual_avg:,.2f} CHF ({(predicted_price/actual_avg - 1)*100:.1f}%)")
    else:
        print("Keine ähnlichen Objekte im Datensatz gefunden.")

## 10. Zusammenfassung und nächste Schritte

**Zusammenfassung:**
- Wir haben verschiedene Modelle für die Immobilienpreisvorhersage implementiert und evaluiert
- Das beste Modell basierend auf R² wurde identifiziert und gespeichert
- Feature Importance-Analyse zeigt, dass Quartier, Zimmeranzahl und Preisniveau die wichtigsten Einflussfaktoren sind
- Manuelle Tests zeigen plausible Vorhersagen für verschiedene Immobilienarten in unterschiedlichen Quartieren

**Nächste Schritte:**
1. Reisezeit-Daten in das Modell integrieren
2. Streamlit-App entwickeln mit interaktiven Karten
3. Modell in die App einbinden
4. Benutzerfreundliche UI erstellen
5. Umfassendes Testing der App
6. Video-Präsentation erstellen